In [2]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [4]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [6]:
model_df = df.copy()

In [7]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [8]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino,"[we, just, wanna, party, party, just, for, you..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino,"[this, is, america, don't, catch, you, slippin..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ..."


In [9]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [10]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [11]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [12]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

0    we just wanna party party just for you we just...
1    this is america dont catch you slippin now don...
2    this is america dont catch you slippin now loo...
3    tell somebody you go tell somebody grandma tol...
4    this is america dont catch you slippin now don...
Name: verse, dtype: object

In [13]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],we just wanna party party just for you we just...,Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...",this is america dont catch you slippin now loo...,Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],tell somebody you go tell somebody grandma tol...,Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...",this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [14]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [15]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],we just wanna party party just for you we just...,Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...",this is america dont catch you slippin now loo...,Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],tell somebody you go tell somebody grandma tol...,Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...",this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [16]:
get_words(model_df)

['we',
 'just',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'you',
 'we',
 'just',
 'want',
 'the',
 'money',
 'money',
 'just',
 'for',
 'you',
 'i',
 'know',
 'you',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'free',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'dance',
 'and',
 'shake',
 'the',
 'frame',
 'we',
 'just',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'you',
 'we',
 'just',
 'want',
 'the',
 'money',
 'money',
 'just',
 'for',
 'you',
 'you',
 'i',
 'know',
 'you',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'free',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'dance',
 'and',
 'shake',
 'the',
 'frame',
 'this',
 'is',
 'america',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'look',
 'what',
 'im',
 'whippin',
 'now',
 'this',
 'is',
 'america',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'dont',
 'catch',
 'you',
 's

In [17]:
verses = list(model_df.verse.values)

In [18]:
verses[:10]

['we just wanna party party just for you we just want the money money just for you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame we just wanna party party just for you we just want the money money just for you you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame',
 'this is america dont catch you slippin now dont catch you slippin now look what im whippin now this is america dont catch you slippin now dont catch you slippin now look what im whippin now',
 'this is america dont catch you slippin now look how im livin now police be trippin now this is america guns in my area word my area i got the strap i gotta carry ima go into this ugh this is guerilla ima go get the bag or ima get the pad im so cold like im so dope like we gon blow like straight up',
 'tell somebody you go tell somebody grandma told me get your money black man get your black man get yo

In [19]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [20]:
count, following = make_word_dictionary(verses)

In [21]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [22]:
verses_bigrams = create_bigrams(verses)

In [23]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [24]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

people that i could convert her cuz i could but rumor has it i got to say to a theater near you around are you aware are you too wild i need cupid


friends first like my iphone you can say whatever that you an angel that shes bubblin in it this ill go around the city aint no bitch all i need someone else call


people pay aiiyo my man i dont take myself serious who the realest that you met me if i didnt have a bye bye you shouldnt have drove tell me where i separate


blue mink rug song this is the last man fucked up royally but now i dropped it and bout it if you didn t say we dont do the same people that love


paper bag daddy lee baca on trial tryna portray a boxer beatin up on me i know its real whats that facts contract max i gotta make it without sayin it was you


red beam we on an everyday basis with poetic justice if i like the new west coasts best kept secret chronic smoke comin out the projects anthony planned to my nebula i will


love it for my city im a fuckin problem i 

love you lovin you 100 proof i came back for you i love you boy should i spend thousands on bottles so i think i got some good head she got the millas


blue tint blue faces get that get that shit back shit nathan peter nico on my laptop screaming at the finish line with the shoes went platinum every time we talked about love


friends would love it when you let me on my ass so fat probably sit the world been down before nigga when she feels obligated admitted it to my boss bitches wifin niggas


people call me the favor let me explain something i cant save name that i felt was more bogus only made it but all three shes got some of these moving units through


club just juice and young lion you know i set it off like wrist slaps uhh dear lord come save me you know what i spit that boy gone you were crazy i


paper is a chemical imbalance in your area will grow into women that i make movies with criminals tryin to get called oreo and faggot i used to what you did you realize


brother b

In [25]:
columns = ['markov_generated_verses']

all_rapper_markov_df = pd.DataFrame(generated_raps, columns=columns)

In [26]:
all_rapper_markov_df['artist'] = 'Combined'

all_rapper_markov_df

,markov_generated_verses,artist
0,people that i could convert her cuz i could bu...,Combined
1,friends first like my iphone you can say whate...,Combined
2,people pay aiiyo my man i dont take myself ser...,Combined
3,blue mink rug song this is the last man fucked...,Combined
4,paper bag daddy lee baca on trial tryna portra...,Combined
5,red beam we on an everyday basis with poetic j...,Combined
6,love it for my city im a fuckin problem i was ...,Combined
7,love no more it wont be moved how used to lick...,Combined
8,cash nice nigga yall knew back when we grow up...,Combined
9,cash old episodes of shining time station you ...,Combined


In [27]:
with open('data/all_rappers_markov_raps.pkl', 'wb') as picklefile:
    pickle.dump(all_rapper_markov_df, picklefile)